In [ ]:
import torch
import pandas as pd
import os
import pickle
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import SimpleITK as sitk

In [ ]:
mount_point = "/mnt/raid/C1_ML_Analysis/"

prediction_sweep = pickle.load(open(os.path.join(mount_point, 'test_output/fetal_biometry/efw/v3.7/efw_2025-11-12_test/prediction_sweep.pkl'), 'rb'))
max_scores_sweep = [torch.max(s).item() for s in prediction_sweep['scores_sweeps']]
efw_pred_max = []
for idx in range(len(prediction_sweep['scores_sweeps'])):
    max_idx = torch.argmax(prediction_sweep['scores_sweeps'][idx])
    efw_pred_max.append(prediction_sweep['efw_pred_frames'][idx].view(-1)[max_idx].item())

In [ ]:
df_test = pd.read_csv('/mnt/raid/C1_ML_Analysis/CSV_files/efw_2025-11-12_test.csv')


In [ ]:
prediction_sweep = pickle.load(open(os.path.join(mount_point, 'test_output/fetal_biometry/efw/v3.7/efw_2025-11-12_test/prediction_sweep.pkl'), 'rb'))
max_scores_sweep = [torch.max(s).item() for s in prediction_sweep['scores_sweeps']]
efw_pred_max = []
for idx in range(len(prediction_sweep['scores_sweeps'])):
    max_idx = torch.argmax(prediction_sweep['scores_sweeps'][idx])
    efw_pred_max.append(prediction_sweep['efw_pred_frames'][idx].view(-1)[max_idx].item())

In [ ]:
df_prediction_sweep = pd.DataFrame({'file_path': prediction_sweep['file_path'], 'study_id': prediction_sweep['id'], 'efw_pred': prediction_sweep['efw_pred']*1000, 'efw_gt': prediction_sweep['efw_gt']*1000, 'max_scores_frames': max_scores_sweep, 'efw_pred_max': np.array(efw_pred_max)*1000})

df_prediction_sweep['efw_error'] = df_prediction_sweep['efw_pred'] - df_prediction_sweep['efw_gt']
df_prediction_sweep['abs_efw_error'] = df_prediction_sweep['efw_error'].abs()
df_prediction_sweep['efw_bin'] = pd.cut(df_prediction_sweep['efw_gt'], bins=[0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000], labels=['0-500','500-1000','1000-1500','1500-2000','2000-2500','2500-3000','3000-3500','3500-4000','4000-4500','4500-5000'])

df_prediction_sweep = df_prediction_sweep.merge(df_test[['study_id', 'dataset']], on='study_id')

In [ ]:
sweeps_error = list(df_prediction_sweep.query('max_scores_frames > 0.8 and abs_efw_error > 500')['file_path'])
print(len(sweeps_error), sweeps_error)

In [ ]:
idx_error = prediction_sweep['file_path'].index(sweeps_error[0])
file_path = prediction_sweep['file_path'][idx_error]
print(file_path)
scores =    prediction_sweep['scores_sweeps'][idx_error].view(-1).numpy()
efw_pred_frames = prediction_sweep['efw_pred_frames'][idx_error].view(-1).numpy()
efw_gt = prediction_sweep['efw_gt'][idx_error]

df_error = pd.DataFrame({'scores': scores, 'efw_pred_frames': efw_pred_frames*1000, 'efw_gt': efw_gt*1000})

In [ ]:
img = sitk.ReadImage(file_path)
img_np = sitk.GetArrayFromImage(img)

px.imshow(img_np, animation_frame=0, binary_string=True, title=f'EFW GT: {efw_gt*1000:.2f}g').show()

In [ ]:
# Scatter plot of scores vs efw_pred_frames
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(df_error)), y=df_error['efw_pred_frames'], name='Predicted EFW'))
fig.add_trace(go.Scatter(x=np.arange(len(df_error)), y=df_error['scores'], name='Scores', yaxis='y2'))
fig.add_trace(go.Scatter(x=np.arange(len(df_error)), y=[efw_gt*1000]*len(df_error), mode='lines', name='EFW GT', line=dict(dash='dash', color='green')))
fig.update_layout(title='EFW Predictions and Scores', xaxis_title='Frame Index', yaxis_title='Predicted EFW (g)', yaxis2=dict(title='Scores', overlaying='y', side='right'))
fig.show()